In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import simple_soil

In [ ]:
results_dict = {
    "total_time": [],
    "iteration": [],
    "error": [],
    "F": [],
    "f": [],
}

In [ ]:
F_t = 0.0
f_t = 0.0
total_time = 0.0
delta_t = 0.25  # hrs
theta_sat = 0.479
psi = 29.22  # cm
K = 0.05  # cm/hr
initial_saturation = 0.30  # fraction
theta_eff = 0.423
delta_F = 1.0e-4

In [ ]:
theta_wp = theta_sat - theta_eff

In [ ]:
theta = initial_saturation * (theta_sat - theta_wp) + theta_wp
theta

In [ ]:
delta_theta = theta_sat - theta
delta_theta

In [ ]:
def residual(F):
    v = abs(psi) * delta_theta
    return F - v * np.log(1.0 + F / v) - K * total_time

In [ ]:
def derivative(F):
    return (residual(F + delta_F) - residual(F)) / delta_F

In [ ]:
def green_ampt_infiltration(F):
    v = abs(psi) * delta_theta
    if F == 0.0:
        f = 0.0
    else:
        f = K * ((v / F) + 1)
    return f

In [ ]:
def fill_results(iteration, error, F_t, f_t):
    results_dict["total_time"].append(total_time)
    results_dict["iteration"].append(iteration)
    results_dict["error"].append(error)
    results_dict["F"].append(F_t)
    results_dict["f"].append(f_t)

In [ ]:
def solve(F_t):
    iter, F_t, error, converged = simple_soil.utils.newton_raphson(
        residual, derivative, F_t
    )
    infiltration_rate = green_ampt_infiltration(F_t)
    fill_results(iter, error, F_t, green_ampt_infiltration(F_t))

In [ ]:
np.arange(0, 2, 0.25)

In [ ]:
for total_time in np.arange(0.0, 3.0 + delta_t, delta_t):
    solve(F_t)

In [ ]:
df = pd.DataFrame.from_dict(
    results_dict,
).set_index("total_time")

In [ ]:
df

In [ ]:
df["f"].plot()

In [ ]:
df["F"].plot()